# Project 13: Movielens 영화 추천 실습

* 추천시스템의 MNIST라고 부를 만한 Movielens 데이터
* 유저가 영화에 대해 평점을 매긴 데이터가 크기 별로 있습니다. MovieLens 1M Dataset 사용.
* 별점 데이터는 explicit 데이터입니다. 그러나 implicit 데이터로 간주하고 테스트 할 수 있습니다. 
* 별점을 시청횟수로 해석합니다.
* 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외합니다.

In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)


1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리

* Movielens 데이터는 rating.dat 안에 인덱싱까지 완료된 사용자-영화-평점 데이터가 정리되어 있습니다. 

In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### MF model 구성
* Matrix Factorization(MF) 모델을 사용합니다. 
* (m,n) 사이즈의 행렬 R을 (m,k)사이즈의 행렬 P와 (k,n)사이즈의 행렬 Q로 분해할수 있습니다. 

* Collaborative Filtering for Implicit Feedback Datasets 논문에서 제안한 모델을 사용합니다.

In [8]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* MF 모델을 implicit 패키지를 사용하여 학습합니다.
* implicit 패키지는 이전 스텝에서 설명한 암묵적 dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지 입니다.
* 이 패키지에 구현된 als(AlternatingLeastSquares)모델을 사용합니다. Matrix Factorization에서 쪼개진 두 feature matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않으므로, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSqures 방식이 효과적인것으로 알려져 있습니다.

* AlternatingLeastSquares의 __init__ 파라미터

    1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
    2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
    3. use_gpu : GPU를 사용할 것인지
    4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [9]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [10]:
# 사용자수
ratings['user_id'].nunique()

6039

In [11]:
# 인기있는 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 선호하는 영화를 5가지 골라서 ratings 에 추가

In [12]:
#  단, 이름은 꼭 데이터셋에 있는 것과 동일하게! 
my_favorite = [ 48, 1,2,39,50]

# 'heawon'이라는 user_id가 위 아티스트의 영화를 30번씩 봤다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'counts':[5,4,3,4,5]})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,48,5,NaN
1,6041,1,4,NaN
2,6041,2,3,NaN
3,6041,39,4,NaN
4,6041,50,5,NaN


In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = movies['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx[1])

6039
0


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings.head()

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,1176,5,978300760.0
1,0,655,3,978302109.0
2,0,902,3,978301968.0
3,0,3339,4,978300275.0
4,0,2286,5,978824291.0


## 4) CSR matrix 만들기

In [18]:
from scipy.sparse import csr_matrix

num_user = len(user_unique)
num_movie = len(movie_unique)


print('num_user:', num_user) # 6040
print('num_movie: ', num_movie) #3628
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))


num_user: 6040
num_movie:  3883


In [36]:
alpha = 40
csr_data = (csr_data * alpha).astype('double')

## 5) als_model = AlternatingLeastSquares 모델 직접 구성하여 훈련하기

In [37]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=10, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)


In [38]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3883x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [39]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

In [40]:
len(user_to_idx)

6040

In [41]:
heawon, toystory = user_to_idx[6041], movie_to_idx[1]
heawon_vector, toystory_vector = als_model.user_factors[heawon], als_model.item_factors[toystory]

print('슝=3')

슝=3


In [42]:
heawon_vector

array([-9.178397,  2.4681  , 22.550537, 32.23937 , 23.186396, 38.04761 ,
       16.332996, 32.636654, 14.32973 ,  6.166132], dtype=float32)

In [43]:
toystory_vector

array([0.00464218, 0.00513277, 0.00501427, 0.00514348, 0.00516092,
       0.00606705, 0.00476111, 0.00565672, 0.00544724, 0.00468124],
      dtype=float32)

In [44]:
np.dot(heawon_vector, toystory_vector)

0.96875995

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [45]:
favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(0, 1.0000001),
 (3045, 0.99999666),
 (2849, 0.9999962),
 (1245, 0.999996),
 (1899, 0.9999948),
 (2918, 0.99999404),
 (33, 0.9999934),
 (352, 0.9999932),
 (2735, 0.99999267),
 (315, 0.99999255),
 (1179, 0.99999243),
 (293, 0.9999916),
 (1081, 0.99999136),
 (49, 0.9999906),
 (1892, 0.99999017)]

In [46]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 2918,
 1265,
 1968,
 2987,
 34,
 356,
 2804,
 318,
 1197,
 296,
 1097,
 50,
 1961]

In [50]:
[movies[movies['movie_id'] == idx_to_movie[i[0]]]['title'].iloc[0] for i in similar_movie]


['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Ferris Bueller's Day Off (1986)",
 'Groundhog Day (1993)',
 'Breakfast Club, The (1985)',
 'Who Framed Roger Rabbit? (1988)',
 'Babe (1995)',
 'Forrest Gump (1994)',
 'Christmas Story, A (1983)',
 'Shawshank Redemption, The (1994)',
 'Princess Bride, The (1987)',
 'Pulp Fiction (1994)',
 'E.T. the Extra-Terrestrial (1982)',
 'Usual Suspects, The (1995)',
 'Rain Man (1988)']

## 8) 내가 가장 좋아할만한 영화를 추천받기

In [52]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2789, 0.97131383),
 (3684, 0.97080517),
 (3509, 0.9706862),
 (1176, 0.9706013),
 (3339, 0.9705413),
 (1959, 0.9705406),
 (2928, 0.9704636),
 (589, 0.97043073),
 (108, 0.9704225),
 (1192, 0.97038245),
 (2693, 0.9703283),
 (476, 0.97019124),
 (3682, 0.9700422),
 (352, 0.9700264),
 (257, 0.96999407),
 (2637, 0.96998096),
 (585, 0.9699694),
 (523, 0.9699414),
 (847, 0.96989596),
 (2614, 0.96984154)]

In [54]:
[movies[movies['movie_id'] == idx_to_movie[i[0]]]['title'].iloc[0] for i in movie_recommended]


['American Beauty (1999)',
 'Patriot, The (2000)',
 'Gladiator (2000)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Erin Brockovich (2000)',
 'Saving Private Ryan (1998)',
 'Being John Malkovich (1999)',
 'Silence of the Lambs, The (1991)',
 'Braveheart (1995)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Sixth Sense, The (1999)',
 'Jurassic Park (1993)',
 'Chicken Run (2000)',
 'Forrest Gump (1994)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'American Pie (1999)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Godfather, The (1972)',
 'Austin Powers: The Spy Who Shagged Me (1999)']

## 회고
* 평소에 넷플릭스나 유튜브에서 추천 동영상이나 추천영화가 뜨는 것을 보았는데, 간단한 버전이지만 신기했습니다!!
* 좋아할만한 영화를 추천받았는데 실제로 좋아했던 영화들이 많이 나왔습니다.